<a href="https://colab.research.google.com/github/Biswajitsahoo29/mentalhealthchatbot/blob/main/friendlychatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit transformers torch librosa gTTS pydub speechrecognition


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import speech_recognition as sr
from gtts import gTTS
import IPython.display as display
import streamlit as st

# Load DialoGPT for AI-generated responses
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Load sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis")

# Store chat history
chat_history_ids = None

# Predefined responses for mental health support
mental_health_responses = {
    "anxious": "It's okay. Everyone feels anxious sometimes. Take deep breaths, and I'm here for you.",
    "stressed": "I understand. Try taking a short break, listening to music, or meditating.",
    "depressed": "You're not alone. I'm here to listen. Talking to a close friend or professional might help.",
    "lonely": "I'm here for you. Maybe reaching out to a loved one could help?",
    "sad": "It's okay to feel sad. Allow yourself to express emotions and do something that makes you happy.",
    "tired": "Make sure you're getting enough rest and drinking water. Taking short breaks can help too!",
    "worried": "I understand. Try to focus on what you can control, and let go of what you can't."
}

def detect_mental_health_issues(user_input):
    user_input = user_input.lower()
    for keyword in mental_health_responses:
        if keyword in user_input:
            return mental_health_responses[keyword]
    return None

def analyze_sentiment(user_input):
    result = sentiment_analyzer(user_input)[0]
    return result['label']

def chatbot_response(user_input):
    global chat_history_ids

    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

def text_to_speech(response):
    tts = gTTS(text=response, lang='en')
    tts.save("response.mp3")
    display.display(display.Audio("response.mp3", autoplay=True))

# Streamlit UI
def main():
    st.title("🧠 Mental Health Chatbot")
    st.write("Type your message below and press Enter to chat.")

    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []

    user_input = st.text_input("You:", "")

    if user_input:
        mental_health_response = detect_mental_health_issues(user_input)

        if mental_health_response:
            response = mental_health_response
        else:
            sentiment = analyze_sentiment(user_input)
            if sentiment == "NEGATIVE":
                response = "I'm here for you. It's okay to feel this way. Do you want to talk about it?"
            else:
                response = chatbot_response(user_input)

        st.session_state.chat_history.append(("You", user_input))
        st.session_state.chat_history.append(("Bot", response))

        for sender, msg in st.session_state.chat_history:
            st.write(f"**{sender}:** {msg}")

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu
2025-02-24 12:19:58.313 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 12:19:58.660 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-24 12:19:58.661 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 12:19:58.662 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 12:19:58.664 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 12:19:58.665 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 12:19:58.666 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-24 12:19:58.667 T

In [ ]:
with open("app.py", "w") as f:
    f.write("""PASTE_YOUR_CODE_HERE""")  # Replace with the full chatbot code above

from google.colab import files
files.download("app.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install streamlit pyngrok
!ngrok authtoken YOUR_NGROK_AUTH_TOKEN
!streamlit run app.py & ngrok http 8501



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
ERROR:  authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.
ERROR:  Your authtoken: YOUR_NGROK_AUTH_TOKEN
ERROR:  Instructions to install your authtoken are on your ngrok dashboard:
ERROR:  https://dashboard.ngrok.com/get-started/your-authtoken
ERROR:  
ERROR:  ERR_NGROK_105
ERROR:  https://ngrok.com/docs/errors/err_ngrok_105
ERROR:  



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.226.203.77:8501

